In [1]:
import os, sys
rmg_path = os.getenv('RMGpy')
# if rmg_path does not have anything, and it is not in sys path, add it to sys path
if rmg_path and rmg_path not in sys.path:
    sys.path.insert(1,rmg_path)  #? 1? Help if looking for multiple?
import os
import re
import numpy as np
from collections import defaultdict, OrderedDict
import pandas as pd
from rmgpy.molecule import Molecule
from rmgpy.reaction import Reaction
import itertools
import IPython
from IPython.display import display, Markdown
# investigate
def mprint(s): display(Markdown(s))

In [8]:
string = "[O]O+C_OO+[CH3]"

r, p = string.split('_')

r1, r2 = r.split("+")
p1, p2 = p.split("+")

r_and_p_SMILES_list = [r1, r2, p1, p2]
for item in r_and_p_SMILES_list:
    
    print Molecule(SMILES=item).toAugmentedInChIKey()

OUUQCZGPVNCOIJ-UHFFFAOYSA-u2
VNWKTOKETHGBQD-UHFFFAOYSA
MHAJPDPJQMAIIY-UHFFFAOYSA
WCYWZMWISLQXQU-UHFFFAOYSA-u1


In [3]:
def isReactantOrProduct(self, reactant):
        """
        Checks to see if a certiant rmgpy.molecule object is in 
        either the reactants side or products side of rmgpy.reaction
        
        Inputs: 
        self (rmgpy.reaction)
        reactant (rmgpy.molecule)
        
        Output:
        result (bool) - True if reactant is in the reaction, Fasle if not.
        """
        
        if reactant in self.reactants:
            gotOne = True
        elif reactant in self.products:
            gotOne = True
        else: 
            gotOne = False
        return gotOne
    
#self = df.iloc[0,1]
#reactant = Molecule(SMILES="OO")
#isReactantOrProduct(self, reactant)

In [4]:
directory = 'C:/users/carle/Code/scratch_test/comparerTST'
results = defaultdict(OrderedDict)
# seems like two data types

# list of all log files
comparerFiles = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

#iterating over the list to count them
k = 0
for fil in comparerFiles:
    if ".log" in fil: # and "updated" in fil: # ask about including 'updated'
        k += 1
results = defaultdict(OrderedDict)

#place to store reactions that are not OOH abstractions
not_ooh_abstraction = []

# most important checkpoints
successKey = 'Normal termination of Gaussian'
failKey = "Error termination"

# for 'index of log file' in range 'all log files'
for i in range(1,k+1):
    r = results[i]
    
    #' {0:d}  '.format( ) ensures placement of integer
    filename = 'AutoTST-comparer.{0:d}.combined.log'.format(i) # TODO change back to 'comparer.updated.' at some point
    filepath = os.path.join(directory,filename)
    
    # reversing and making sure log file exists
    if os.path.exists(filepath):
        r['1 log file exists'] = 1
    else:
        continue
    
    # using with to open file, read contents, then close file
    with open(filepath) as f:
        lines = f.readlines()
    
    # j is iterator, while l is the line - enumerate splits these up
    for j,l in enumerate(lines):
        m = re.match('comparerTST.py:177 performCalcs INFO reaction: (.*)', l)
        
        # if it 'performCalcs INFO reaction'
        if m:
            # "first parenthesized sub-group"
            rxn = eval(m.group(1))
            
            # checking it is OOH abstraction
            if not (isReactantOrProduct(rxn, Molecule(SMILES="[O]O")) and isReactantOrProduct(rxn, Molecule(SMILES="OO"))): #Checking to make sure that 
                not_ooh_abstraction.append(i) 
                break
            r['0 reaction'] = rxn      
        
        # adding 'checkpoints to r dict if they exist, changing around checkpoint name too'
        if 'We have generated a H_Abstraction reaction that matches, and used it to label the atoms' in l:
            r['2 matched H-abstraction'] = 1
          
        if 'Reading existing kinetics file' in l:
            r['XX using existing kinetics data file'] = 1 # Reading in existing .kinetics file
             
        if 'Generating a TS geometry via the direct guess method' in l:
            r['3A started making TS geometry'] = 1
        if "Reading existing ts file" in l:
            r['3B using existing ts data file'] = 1 # If reading in existing .ts file, this bypasses 4, 5 and 6
        
        #3C
        if 'optimizeTS INFO Output file' and 'exists and looks complete. Trying that.' in l:
            r['3C Previous TS optimization complete'] = 1 #If reading in existing .log, this then checks if there is an existing IRC calc   
        #
        
        if 'Running loose optimization of TS with frozen center' in l:
            r['4A TS opt w frozen center'] = 1    
        if 'Optimization of TS reaction center distances' in l:
            r['4B TS opt of rxn center'] = 1  
        if 'Optimizing TS attempt' in l:
            r['4C TS optimization started'] = 1
        
        #5A
        # 'verifying...' prints whenever the file is being verified
        # do you want 5A=1 if TS is complete but does not have all the keywords for success?
        if 'verifyOutputFile INFO Verifying output file' in l:
            r['5A New TS optimization complete'] = 1
        #    
        
        if 'Creating IRC file' in l:
            r['6A IRC file created'] = 1
        if "Verifying the IRC output file" in l:
            r['6B New IRC calc complete'] = 1
        if "saveTSData INFO Saving TS result file" in l:
            r['6C New IRC calc successful'] = 1   
            
        if 'Symmetry input file written to' in l:
            r['7A starting Symmetry calculation'] = 1
        if 'Point group:' in l:
            r['7B Symmetry calc successful'] = 1
         
        if 'CanTherm execution initiated' in l:
            r['8 CanTherm started'] = 1
        if 'One or both of the barrier heights of' in l:
            r['8A CanTherm barrier height problem'] = -1
         
        if 'Yay, reaction kinetics calculated!!!' in l:
            r['ZZ overall success'] = 1
            r['ZZZ Complete'] = 1
            
        if "Boo," in l:
            r['ZZ overall fail'] = 1
            r['ZZZ Complete'] = 1
        
        if "gaussian.py:880 verifyIRCOutputFile ERROR Not all of the required keywords for success were found in the IRC output file!" in l:
            r['ZZZ IRC success keywords not found'] = 1

    # 'checkpoints' that require multiple other checkpoints
    if 'XX using existing TS data file' and 'ZZ overall success' in r.keys():
        r['YY successful prior calculation'] = 1
        
    if '5A New TS optimization complete' and '6A IRC file created' in r.keys():
        r['5B TS successfully optimized'] = 1
    
    if '6B New IRC calc complete' in r.keys() and "6A IRC file created" not in r.keys():
        r['6D IRC calc from previous calculation'] = 1
        del(r['6B New IRC calc complete'])
        
    if '5A New TS optimization complete' and '5C Previous TS optimization complete' in r.keys():
        del(r['5A New TS optimization complete'])
    
    """
    if '3C Previous TS optimization complete' in r.keys():
            
        qmScratchDir = '/gss_gpfs_scratch/harms.n/QMscratch/'

        fileNames = [f for f in os.listdir(qmScratchDir) if os.path.isfile(os.path.join(qmScratchDir, f))]

        r1, r2 = eval(rxn).reactants
        p1, p2 = eval(rxn).products
        

        r1SMILES = r1.toSMILES()
        
        r2SMILES = r2.toSMILES()
        p1SMILES = p1.toSMILES()
        p2SMILES = p2.toSMILES()
        
        r1Augmented = r1.toInChIKey()
        r2Augmented = r2.toInChIKey()
        p1Augmented = p1.toInChIKey()
        p2Augmented = p2.toInChIKey()
        
        for fileName in fileNames:
            if r1Augmented or r2Augmented or p1Augmented or p2Augmented in fileName:
                reactOrProdFilePath = os.path.join(qmScratchDir, fileName)
                f = open(reactOrProdFilePath, "r")
                lastLines = f.readlines()
                    
                if sucessKey or failKey in lastLines:
                    r['WW.0 Complete Reactants / Products Estimate'] = 1
                if failKey in lastLines:
                    r['WW.1 Fail Reactants / Products Estimate'] = 1 
                
            if r1SMILES and r2SMILES and p1SMILES and p2SMILES and ".log" in fileName:
                if "Est" in fileName:
                    est = os.path.join(qmScratchDir, fileName)
                    f = open(est, "r")
                    lastLines = f.readlines()
                    #f.closed
                    
                    if sucessKey or failKey in lastLines:
                        r['WW.A Complete TS Estimate'] = 1
                    if sucessKey in lastLines:
                        r['WW.B Successful TS Estimate'] = 1
                             
                    # check complete
                elif "RxnC" in fileName:
                    rxnC = os.path.join(qmScratchDir, fileName)
                    f = open(rxnC, "r")
                    lastLines = f.readlines()
                    #f.closed()
                    
                    if sucessKey or failKey in lastLines:
                        r['WW.C Complete Rxn Center'] = 1
                    if sucessKey in lastLines:
                        r['WW.D Successful Rxn Center'] = 1
                    # check complete
                    
                elif "IRC" in fileName:
                    IRC = os.path.join(qmScratchDir, fileName)
                    f = open(IRC, "r")
                    lastLines = f.readlines()
                    #f.closed()
                    
                    if sucessKey or failKey in lastLines:
                        r['WW.G Complete IRC log'] = 1
                    if sucessKey in lastLines:
                        r['WW.H Successful IRC log'] = 1
                    # check complete
                    
                else:
                    overall = os.path.join(qmScratchDir, fileName)
                    f = open(overall, "r")
                    lastLines = f.readlines()[-4:]
                    #f.closed()
                    
                    if sucessKey or failKey in lastLines:
                        r['WW.E Complete overall TS log'] = 1
                    if sucessKey in lastLines:
                        r['WW.F Successful overall TS log'] = 1
                    # check complete
                    """
                    
                    

               
# remobing non OOH absraction reactions
for i in not_ooh_abstraction:
    del(results[i])

# creating pandas data frame (keys as row titles)
df = pd.DataFrame(results)

#printing DF
df

,5,7,8,12,13,14,16,17,21,22,...,456,457,458,459,460,461,462,463,464,465
0 reaction,"<Molecule ""OO""> + <Molecule ""[H]""> <=> <Molecu...",NaN,"<Molecule ""[CH3]""> + <Molecule ""[O]O""> <=> <Mo...","<Molecule ""[CH3]""> + <Molecule ""OO""> <=> <Mole...","<Molecule ""C""> + <Molecule ""[O]O""> <=> <Molecu...","<Molecule ""OO""> + <Molecule ""[O]""> <=> <Molecu...","<Molecule ""[O]O""> + <Molecule ""O""> <=> <Molecu...","<Molecule ""OO""> + <Molecule ""[OH]""> <=> <Molec...","<Molecule ""C#C""> + <Molecule ""[O]O""> <=> <Mole...","<Molecule ""[O]O""> + <Molecule ""C#C""> <=> <Mole...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 log file exists,1,NaN,1,1,1,1,1,1,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2 matched H-abstraction,1,NaN,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3A started making TS geometry,NaN,NaN,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3B using existing ts data file,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3C Previous TS optimization complete,NaN,NaN,NaN,1,1,1,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4A TS opt w frozen center,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4B TS opt of rxn center,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4C TS optimization started,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5A New TS optimization complete,NaN,NaN,1,1,1,1,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
rxn = df.T["0 reaction"].loc[13]
rxn

Cairo not found; molecule will not be drawn.


IOError: [Errno 2] No such file or directory: 'temp_reaction.png'

Reaction(reactants=[Molecule(SMILES="C"), Molecule(SMILES="[O]O")], products=[Molecule(SMILES="OO"), Molecule(SMILES="[CH3]")], degeneracy=4.0, pairs=[[Molecule(SMILES="C"), Molecule(SMILES="[CH3]")], [Molecule(SMILES="[O]O"), Molecule(SMILES="OO")]])

In [119]:
successKey = 'Normal termination of Gaussian' #, '******    1 imaginary frequencies (negative Signs) ******',]

failKey = 'Error termination' #, 'Error in internal coordinate system.',]

if '3C Previous TS optimization complete' in df.index:
            
    qmScratchDir = 'C:/users/carle/Code/scratch_test/scratch'

    fileNames = [f for f in os.listdir(qmScratchDir) if os.path.isfile(os.path.join(qmScratchDir, f))]
    
    r1, r2 = rxn.reactants
    p1, p2 = rxn.products


    r1SMILES = r1.toSMILES()

    r2SMILES = r2.toSMILES()
    p1SMILES = p1.toSMILES()
    p2SMILES = p2.toSMILES()

    r1Augmented = r1.toInChIKey()
    r2Augmented = r2.toInChIKey()
    p1Augmented = p1.toInChIKey()
    p2Augmented = p2.toInChIKey()

    allAugInchis = (r1Augmented, r2Augmented, p1Augmented, p2Augmented)
    
    def checkAugInchi(pAugInch, filename, mastList):
        reactOrProdFilePath = os.path.join(qmScratchDir, fileName)
        f = open(reactOrProdFilePath, "r")
            
        lines = f.readlines()

        if successKey or failKey in lines:
            r['WW.0 Complete Reactants / Products Estimate'] = 1
        if failKey in lines:
            r['WW.1 Fail Reactants / Products Estimate'] = 1
            
        mastList.append(r)
        return

    def checkSmileFile(filename, mastList):        
        # shell of reaction
        if "Est" in fileName:
            est = os.path.join(qmScratchDir, fileName)
            f = open(est, "r")
            lines = f.readlines()
            #f.closed

            if successKey or failKey in lines:
                r['WW.A Complete TS Estimate'] = 1
            if successKey in lines:
                r['WW.B Successful TS Estimate'] = 1

        # Reaction Center
        elif "RxnC" in fileName:
            rxnC = os.path.join(qmScratchDir, fileName)
            f = open(rxnC, "r")
            lines = f.readlines()
            #f.closed()

            if successKey or failKey in lines:
                r['WW.C Complete Rxn Center'] = 1
            if successKey in lines:
                r['WW.D Successful Rxn Center'] = 1
            # check complete

        #walking down slope
        elif "IRC" in fileName:
            IRC = os.path.join(qmScratchDir, fileName)
            f = open(IRC, "r")
            lines = f.readlines()
            #f.closed()

            if successKey or failKey in lines:
                r['WW.G Complete IRC log'] = 1
            if successKey in lines:
                r['WW.H Successful IRC log'] = 1
            # check complete


        else:
            overall = os.path.join(qmScratchDir, fileName)
            f = open(overall, "r")
            lines = f.readlines()[-4:]
            #f.closed()

            if successKey or failKey in lines:
                r['WW.E Complete overall TS log'] = 1
            if successKey in lines:
                r['WW.F Successful overall TS log'] = 1
            # check complete
        
        mastList.append(r)
        return

    def possibleNames(r1SMILES, r2SMILES, p1SMILES, p2SMILES):
            reactants = (r1SMILES, r2SMILES)
            products = (p1SMILES, p2SMILES)
            
            joinedReactOrders = []
            joinedProdOrders = []

            for order in itertools.permutations(reactants):
                joinedReactOrders.append('+'.join(order))

            for order in itertools.permutations(products):
                joinedProdOrders.append("+".join(order))

            reactions = ['_'.join( (rJO, pJO) ) for rJO in joinedReactOrders for pJO in joinedProdOrders]
            
            return reactions
    
    # iterating over qm scratch files
    mastRs =[]
    smileFileNames = []
    augFileNames = {}
    
    for fileName in fileNames:

#TODO change to list comprehension method
        for smilesReaction in possibleNames(r1SMILES, r2SMILES, p1SMILES, p2SMILES):
            
            if smilesReaction in fileName:
                checkSmileFile(fileName, mastRs)
                smileFileNames.append(fileName)
            
        for augP in augPs:
            
            if augP in fileName:
                checkAugInchi(augP, fileName, mastRs)
                augFileNames[augP] = fileName
            
print

['C+[O]O_OO+[CH3]', 'C+[O]O_[CH3]+OO', '[O]O+C_OO+[CH3]', '[O]O+C_[CH3]+OO']



In [121]:
#! finding Carbons in 'RxnC.log'
#! finding 'OO' in augInchi files
#! no files for augInchi style


print smilePs
print possibleNames(r1SMILES, r2SMILES, p1SMILES, p2SMILES)
print
print smileFileNames
print
print augPs
print 
print augFileNames
print 
print 

('C', '[O]O', 'OO', '[CH3]')
['C+[O]O_OO+[CH3]', 'C+[O]O_[CH3]+OO', '[O]O+C_OO+[CH3]', '[O]O+C_[CH3]+OO']

[]

('VNWKTOKETHGBQD-UHFFFAOYSA', 'OUUQCZGPVNCOIJ-UHFFFAOYSA', 'MHAJPDPJQMAIIY-UHFFFAOYSA', 'WCYWZMWISLQXQU-UHFFFAOYSA')

{}


In [ ]:
successKey = 'Normal termination of Gaussian' #, '******    1 imaginary frequencies (negative Signs) ******',]

failKey = 'Error termination' #, 'Error in internal coordinate system.',]

if '3C Previous TS optimization complete' in df.index:
            
    qmScratchDir = 'C:/users/carle/Code/scratch_test/scratch'

    fileNames = [f for f in os.listdir(qmScratchDir) if os.path.isfile(os.path.join(qmScratchDir, f))]
    
    r1, r2 = rxn.reactants
    p1, p2 = rxn.products


    r1SMILES = r1.toSMILES()

    r2SMILES = r2.toSMILES()
    p1SMILES = p1.toSMILES()
    p2SMILES = p2.toSMILES()

    r1Augmented = r1.toInChIKey()
    r2Augmented = r2.toInChIKey()
    p1Augmented = p1.toInChIKey()
    p2Augmented = p2.toInChIKey()

    # iterating over qm scratch files
    for fileName in fileNames:
        
        # opening if augmented inchi in filename
        if r1Augmented or r2Augmented or p1Augmented or p2Augmented in fileName:

            reactOrProdFilePath = os.path.join(qmScratchDir, fileName)
            f = open(reactOrProdFilePath, "r")
            
            lines = f.readlines()

            if successKey or failKey in lines:
                r['WW.0 Complete Reactants / Products Estimate'] = 1
            if failKey in lines:
                r['WW.1 Fail Reactants / Products Estimate'] = 1
                
       
        if r1SMILES and r2SMILES and p1SMILES and p2SMILES and ".log" in fileName:
            
            # shell of reaction
            if "Est" in fileName:
                est = os.path.join(qmScratchDir, fileName)
                f = open(est, "r")
                lines = f.readlines()
                #f.closed

                if successKey or failKey in lines:
                    r['WW.A Complete TS Estimate'] = 1
                if successKey in lines:
                    r['WW.B Successful TS Estimate'] = 1

                # check complete
                
            # Reaction Center
            elif "RxnC" in fileName:
                rxnC = os.path.join(qmScratchDir, fileName)
                f = open(rxnC, "r")
                lines = f.readlines()
                #f.closed()

                if successKey or failKey in lines:
                    r['WW.C Complete Rxn Center'] = 1
                if successKey in lines:
                    r['WW.D Successful Rxn Center'] = 1
                # check complete
            
            #walking down slope
            elif "IRC" in fileName:
                IRC = os.path.join(qmScratchDir, fileName)
                f = open(IRC, "r")
                lines = f.readlines()
                #f.closed()

                if successKey or failKey in lines:
                    r['WW.G Complete IRC log'] = 1
                if successKey in lines:
                    r['WW.H Successful IRC log'] = 1
                # check complete
            
    
            else:
                overall = os.path.join(qmScratchDir, fileName)
                f = open(overall, "r")
                lines = f.readlines()[-4:]
                #f.closed()

                if successKey or failKey in lines:
                    r['WW.E Complete overall TS log'] = 1
                if successKey in lines:
                    r['WW.F Successful overall TS log'] = 1
                # check complete


In [27]:
r

OrderedDict([('1 log file exists', 1),
             ('WW.0 Complete Reactants / Products Estimate', 1),
             ('WW.E Complete overall TS log', 1),
             ('WW.A Complete TS Estimate', 1),
             ('WW.C Complete Rxn Center', 1),
             ('WW.G Complete IRC log', 1)])

In [33]:
print rxn
print f

<Molecule "C"> + <Molecule "[O]O"> <=> <Molecule "OO"> + <Molecule "[CH3]">
<open file 'C:/users/carle/Code/scratch_test/scratch\\[O]O+[O]O_OO+[O][O]RxnC.log', mode 'r' at 0x000000000DAFE150>
